Project Objective:

The objective of this mini project is to develop skills in web scraping by extracting and analyzing car details from Cars24.com. Interns will focus on gathering data specific to the Mumbai location, covering key details such as kilometers driven, year of manufacture, fuel type, transmission, and price. By the end of this project, interns will have hands-on experience in web scraping, data cleaning, and presenting structured data for analysis.


Project Requirements:

1) Scraping Target: Cars listed on Cars24.com from 3 locations.


2) Data to Collect:

Kilometers Driven

Year of Manufacture

Fuel Type

Transmission

Price

Location

Number of owners

In [1]:
link = "https://www.cars24.com/buy-used-audi-cars-chennai/"
print(link)

https://www.cars24.com/buy-used-audi-cars-chennai/


In [2]:
import requests

page = requests.get(link)
print(page)

<Response [200]>


In [3]:
from bs4 import BeautifulSoup as bs

In [4]:
page_html = bs(page.content, "html.parser")

In [5]:
cars = page_html.find_all('div', {"class" : "styles_outer__NTVth"})

In [6]:
len(cars)

3

In [7]:
from selenium import webdriver
import time

In [ ]:
page_html = bs(html, "html.parser")
cars = page_html.find_all('div', {"class" : "styles_outer__NTVth"})

In [ ]:
len(cars)

In [ ]:
car = cars[1]

In [ ]:
print(car.prettify())

In [ ]:
name = car.find('span').get_text()
print(name)

In [ ]:
manufacture_year = name.split()[0]
brand_name = name.split()[1]
car_model = " ".join(name.split()[1:])

In [ ]:
print(manufacture_year)
print(brand_name)
print(car_model)

In [ ]:
product = car.find('div').get_text()

In [17]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

# List of cities
cities = ["mumbai", "chennai", "bangalore"]

rows = []

# Selenium options (optional: run headless)
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

for city in cities:
    print(f"\n================ {city.title()} Cars ================\n")
    
    url = f"https://www.cars24.com/buy-used-audi-cars-{city}/"

    # Initialize browser
    browser = webdriver.Chrome(options=options)
    
    try:
        browser.get(url)
        time.sleep(3)

        # Scroll until no new cars load
        ini_height = browser.execute_script("return document.body.scrollHeight")

        while True:
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == ini_height:
                break
            ini_height = new_height

        html = browser.page_source
    except Exception as e:
        print(f"⚠️ Failed to load {city}: {e}")
        browser.quit()
        continue

    browser.quit()

    # Parse page with BeautifulSoup
    soup = bs(html, "html.parser")
    cars = soup.find_all('div', {"class" : "styles_outer__NTVth"})
    print(f"Total Cars Found: {len(cars)}")

    for car in cars:
        # Safe extraction with checks
        name_tag = car.find('span', {'class': "sc-braxZu kjFjan"})
        if not name_tag:
            continue

        name = name_tag.get_text(strip=True)
        manufacture_year = name.split()[0] if name else "N/A"
        brand_name = name.split()[1] if len(name.split()) > 1 else "N/A"
        car_model = " ".join(name.split()[1:]) if len(name.split()) > 1 else "N/A"

        details = car.find_all('p', {'class': "sc-braxZu kvfdZL"})
        kilometers_driven = details[0].get_text(strip=True) if len(details) > 0 else "N/A"
        fuel_type = details[1].get_text(strip=True) if len(details) > 1 else "N/A"
        transmission = details[2].get_text(strip=True) if len(details) > 2 else "N/A"
        reg_no = details[3].get_text(strip=True) if len(details) > 3 else "N/A"
        owners = details[4].get_text(strip=True) if len(details) > 4 else "N/A"

        price_tag = car.find('p', {'class': "sc-braxZu cyPhJl"})
        price = price_tag.get_text(strip=True) if price_tag else "N/A"

        location = city.title()  # Use loop variable, not page

        # Append row
        rows.append({
            "Car Name": name,
            "Brand": brand_name,
            "Model": car_model,
            "Kilometers Driven": kilometers_driven,
            "Year of Manufacture": manufacture_year,
            "Fuel Type": fuel_type,
            "Transmission": transmission,
            "Price": price,
            "Location": location,
            "Registration No": reg_no,
            "Number of Owners": owners
        })

# Save to CSV with correct encoding for ₹ symbol
df = pd.DataFrame(rows)
df.to_csv("Audi_webScrapping.csv", index=False, encoding="utf-8-sig")
print(f"\n✅ Done — saved {len(df)} records to Audi_webScrapping.csv")



================ Mumbai Cars ================

Total Cars Found: 12

================ Chennai Cars ================

Total Cars Found: 3

================ Bangalore Cars ================

Total Cars Found: 12

✅ Done — saved 27 records to Audi_webScrapping.csv


In [15]:
df.head()

,Car Name,Brand,Model,Kilometers Driven,Year of Manufacture,Fuel Type,Transmission,Price,Location,Registration No,Number of Owners
0,2010 Audi A4,Audi,Audi A4,12.35k km,2010,Petrol,Auto,₹4.00 lakh,Mumbai,MH-01,N/A
1,2010 Audi Q5,Audi,Audi Q5,99.56k km,2010,Petrol,Auto,₹5.13 lakh,Mumbai,MH-05,N/A
2,2013 Audi A6,Audi,Audi A6,1.3L km,2013,Diesel,Auto,₹12.30 lakh,Mumbai,MH-01,N/A
3,2011 Audi A4,Audi,Audi A4,48.18k km,2011,Petrol,Auto,₹5.36 lakh,Mumbai,MH-01,N/A
4,2011 Audi A6,Audi,Audi A6,1.3L km,2011,Diesel,Auto,₹6.55 lakh,Mumbai,MH-04,N/A


In [16]:
df.tail()

,Car Name,Brand,Model,Kilometers Driven,Year of Manufacture,Fuel Type,Transmission,Price,Location,Registration No,Number of Owners
22,2013 Audi Q5,Audi,Audi Q5,1.7L km,2013,Diesel,Auto,₹12.82 lakh,Bangalore,KA-03,N/A
23,2018 Audi A6,Audi,Audi A6,1.0L km,2018,Diesel,Auto,₹25.00 lakh,Bangalore,KA-01,N/A
24,2013 Audi Q3,Audi,Audi Q3,1.1L km,2013,Diesel,Auto,₹10.85 lakh,Bangalore,KA-41,N/A
25,2014 Audi A3,Audi,Audi A3,1.2L km,2014,Diesel,Auto,₹13.00 lakh,Bangalore,KA-51,N/A
26,2011 Audi Q5,Audi,Audi Q5,1.1L km,2011,Diesel,Auto,₹10.00 lakh,Bangalore,KA-51,N/A
